In [ ]:
# Import smtplib for the actual sending function
import smtplib
import os
import pandas as pd
from datetime import datetime, timedelta

# Import the email modules we'll need
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
from email.mime.base import MIMEBase
from email.encoders import encode_base64

In [ ]:
def check_ext(f, target_ext):
    return True if (os.path.splitext(f)[1] == target_ext) else False
def check_sale_reports(f):
    return True if 'rpt_S8_Item_summ' in f else False
def check_inventory_reports(f):
    return True if '' in f else False
def matches(f, date):
    return True if date in f else False

reports = list(filter(lambda f: check_ext(f, '.xls'), os.listdir('/Users/nathansuberi/Downloads/')))
sale_reports = list(filter(check_sale_reports, reports))
inventory_reports = list(filter(check_inventory_reports, reports))

sale_report_dates = [datetime.strptime(f[:8], '%d%m%Y') for f in sale_reports]
recent_sale_report_date = sorted(sale_report_dates, reverse=True)[0].strftime('%d%m%Y')
recent_sale_report = list(filter(lambda f: matches(f, recent_sale_report_date), sale_reports))

inventory_report_dates = [datetime.strptime(f[:8], '%d%m%Y') for f in inventory_reports]
recent_inventory_report_date = sorted(inventory_report_dates, reverse=True)[0].strftime('%d%m%Y')
recent_inventory_report = list(filter(lambda f: matches(f, recent_inventory_report_date), inventory_reports))

In [ ]:
FROM = os.environ.get('reports_user')
PASS = os.environ.get('reports_password')
SOLD_REPORT = recent_sale_report
INVENTORY_REPORT = recent_inventory_report
DATE_FORMAT = '%m-%d-%Y'
PADRES = 'nsuberi@gmail.com'

In [ ]:
vendors = {
    'ORT': {'company_name':'Vionic','rep_email':'nsuberi@gmail.com','rep_name':'Andrew'}, 
    'DAN': {'company_name':'Dansko','rep_email':'nsuberi@gmail.com','rep_name':'Cliff'},
    'NAO': {'company_name':'Naot','rep_email':'nsuberi@gmail.com','rep_name':'Bruce'},
    'MEP': {'company_name':'Mephisto','rep_email':'nsuberi@gmail.com','rep_name':''},
    'OLU': {'company_name':'Olukai','rep_email':'nsuberi@gmail.com','rep_name':''},
    'AET': {'company_name':'Aetrex','rep_email':'nsuberi@gmail.com','rep_name':''},
    'PIK': {'company_name':'Pikolinos','rep_email':'nsuberi@gmail.com','rep_name':''},
    'TAO': {'company_name':'Taos','rep_email':'nsuberi@gmail.com','rep_name':''},
    'HAF': {'company_name':'Haflinger','rep_email':'nsuberi@gmail.com','rep_name':''},
    'FIN': {'company_name':'Finn Comfort, Think','rep_email':'nsuberi@gmail.com','rep_name':''},
    'SIM': {'company_name':'Simco','rep_email':'nsuberi@gmail.com','rep_name':''}
}

In [ ]:
sold_data = pd.read_excel(SOLD_REPORT)
sold_data = sold_data.reset_index()
sold_data = sold_data.dropna(axis=0, how='all')
sold_data = sold_data.drop([1,6,8,10,15])
sold_data.columns = sold_data.loc[12]
sold_data = sold_data.drop([12])
sold_data = sold_data.dropna(axis=1, how='all')
sold_data.columns = ['DCS Code',
 'Vendor\nCode',
 'Description1',
 None,
 'Attribute',
 None,
 'Size',
 'UPC',
 'QTY',
 None,
 'Ext Price',
 'Ext Cost',
 'Ext Margin Amount',
 'Margin %']
#sold_data.columns = [name if type(name) == str else None for name in sold_data.columns]
sold_data = sold_data.drop(None, axis=1)

In [ ]:
inventory_data = pd.read_excel(INVENTORY_REPORT)

inventory_data = inventory_data[['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 8', 'Unnamed: 13', 'Unnamed: 17', 'Unnamed: 24', 'Unnamed: 27', 'Unnamed: 31']]
inventory_data.columns = ['DCS', 'Vend Code', 'Desc1', 'Attr', 'Size', 'Qty', 'Price', 'Cost']
inventory_data = inventory_data.dropna(axis=0, how='all')
inventory_data = inventory_data.dropna(axis=1, how='all')
inventory_data = inventory_data.drop(16)

In [ ]:
server = smtplib.SMTP('smtp.gmail.com:587')
server.ehlo()
server.starttls()
server.login(FROM, PASS)

start_date = datetime.today() - timedelta(days=7)
end_date = datetime.today() - timedelta(days=1)

for vendor, info in vendors.items():
    
    msg = MIMEMultipart()
    msg['Subject'] = '{} sales during period {} to {}, current inventory'.format(info['company_name'], start_date.strftime(DATE_FORMAT), end_date.strftime(DATE_FORMAT))
    msg['From'] = FROM
    msg['To'] = COMMASPACE.join([info['rep_email'], PADRES])
    msg['Date'] = formatdate(localtime=True)
    
    text = MIMEText('Hi, {}! Attached are our sales for {} in the past week. Could you please take a look and give us some recommendations on a PO based on our current sell through? Thanks! Please let me know what frequency you would like to receive this message.'.format(info['rep_name'], info['company_name']), 'plain', 'us-ascii')
    msg.attach(text)
    
    # Create attachments by subsetting the sold_data and inventory_data
    sold_in_vendor = sold_data[sold_data['Vendor\nCode'] == vendor].sort_values(by=['Description1', 'Attribute', 'Size'])
    inventory_in_vendor = inventory_data[inventory_data['Vend Code'] == vendor].sort_values(by=['Desc1', 'Attr', 'Size'])

    sold_in_vendor.to_csv('tmp_sold.csv')
    inventory_in_vendor.to_csv('tmp_inv.csv')
    
    for d in ['sold', 'inv']:
        with open('tmp_{}.csv'.format(d), 'rb') as data:
            csv = MIMEBase('application', 'octet-stream')
            csv.set_payload(data.read())
            encode_base64(csv)
            csv.add_header('Content-Disposition', 'attachment',
                              filename=os.path.basename('tmp_{}.csv'.format(d)))
            msg.attach(csv)
        os.remove('tmp_{}.csv'.format(d))
            
    
    # Send the message via our own SMTP server.
    server.send_message(msg)

server.quit()